# Modelos predictivos para el trabajo de SIT1

In [2]:
import pandas as pd

In [3]:
def load_data():
    # para la version de prototipo tenemos los datos
    # en el mismo directorio
    data = pd.read_csv('risk_factors_cervical_cancer.csv', sep=',')
    # separar los targets: Hinselmann, Schiller, Citology, Biopsy
    labels = {
        'Hinselmann': data['Hinselmann'],
        'Schiller': data['Schiller'],
        'Citology': data['Citology'],
        'Biopsy': data['Biopsy']
    }
    # elimina los targets del dataframe
    del data['Hinselmann']
    del data['Schiller']
    del data['Citology']
    del data['Biopsy']
    return data, labels